In [1]:
import pandas as pd
import glob
import math
import numpy as np

from helpers import *

v_data='v2'
v_model='m04_4f'

n_files=4

# Get the validation files

In [2]:
data_path = 'data/' # TODO: point this to your data folder
#submission_path = data_path + 'submissions/'
training_path = data_path + 'training_set/'
#fin_X = sorted(glob.glob(training_path + "X_val*.csv.gz")) # TODO: point this to a subset of csv's in your training set
#fin_y = sorted(glob.glob(training_path + "y_val*.csv.gz"))
fins = sorted(glob.glob(training_path + "outDD_"+v_data+"_1*.h5"))
ys   = sorted(glob.glob(training_path + "y_1*_.h5"))

In [3]:
fins

['data/training_set/outDD_v2_10.h5',
 'data/training_set/outDD_v2_11.h5',
 'data/training_set/outDD_v2_12.h5',
 'data/training_set/outDD_v2_13.h5',
 'data/training_set/outDD_v2_14.h5',
 'data/training_set/outDD_v2_15.h5',
 'data/training_set/outDD_v2_16.h5',
 'data/training_set/outDD_v2_17.h5',
 'data/training_set/outDD_v2_18.h5',
 'data/training_set/outDD_v2_19.h5']

In [4]:
ys

['data/training_set/y_10_.h5',
 'data/training_set/y_11_.h5',
 'data/training_set/y_12_.h5',
 'data/training_set/y_13_.h5',
 'data/training_set/y_14_.h5',
 'data/training_set/y_15_.h5',
 'data/training_set/y_16_.h5',
 'data/training_set/y_17_.h5',
 'data/training_set/y_18_.h5',
 'data/training_set/y_19_.h5']

# Read in the data

In [5]:
df_trn = pd.concat([read_log(f) for f in fins[:n_files]], axis=0, ignore_index=True)
df_trn.rename({'not_skipped': 'skip_4'}, axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder
df_trn['session_id'] = LabelEncoder().fit_transform(df_trn['session_id'])
df_trn['session_id'] = df_trn['session_id'].astype(np.uint32)
gc.collect()

n_trk=5
X_trn, y_trn, X_trk = get_XY(df_trn, aggs, reset_index=True, 
                  list_musik_qualities_=list_musik_qualities,
                  aggs_music_qualities_=aggs_music_qualities,
                   i_=list(range(n_trk)))

del df_trn
gc.collect()

651

In [6]:
fins[:n_files]

['data/training_set/outDD_v2_10.h5',
 'data/training_set/outDD_v2_11.h5',
 'data/training_set/outDD_v2_12.h5',
 'data/training_set/outDD_v2_13.h5']

# Get the ground truth

In [7]:
#ground_truth = pd.Series(get_y_truth(df_trn))
ground_truth = pd.concat([pd.read_hdf(f, key='df') for f in ys[:n_files]], axis=0)

In [8]:
ys[:n_files]

['data/training_set/y_10_.h5',
 'data/training_set/y_11_.h5',
 'data/training_set/y_12_.h5',
 'data/training_set/y_13_.h5']

In [9]:
ground_truth.head()

0    [1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
1                   [1, 1, 1, 1, 0]
2    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
3             [0, 0, 0, 0, 0, 1, 1]
4    [1, 0, 0, 0, 0, 1, 0, 1, 0, 0]
dtype: object

# Make predictions using pre-trained models

In [10]:
    l_prob = []
    for i_ in tqdm(list(range(n_trk))):
        X = pd.concat([X_trn, X_trk[i_]], axis=1)
        if len(l_prob) > 0:
            X = pd.concat([X, pd.DataFrame({'pred_trk{}'.format(j): l_prob[j] 
                                            for j in range(len(l_prob))
                                           })],
                          axis=1)
#         display(X.head())
        import joblib
        mfn = 'models/model_{}_{}_{}.pkl'.format(v_data, v_model, i_)
#         print(mfn)
        mdl = joblib.load(mfn)

        prob_pred = mdl.predict_proba(X)[:,1]

        l_prob.append(prob_pred)
        
    # transform accumulated predicted probabilities into competition predictions           
    preds = pred_series_of_lists([p>0.50 for p in l_prob], ground_truth.apply(len), i_2fill=-2)

100%|██████████| 5/5 [00:29<00:00,  5.92s/it]


In [11]:
preds.head(20)

0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1                    [1, 1, 1, 1, 0]
2     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3              [0, 0, 0, 0, 0, 0, 0]
4     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
5                    [0, 0, 0, 0, 0]
6     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
7              [1, 1, 1, 1, 1, 1, 1]
8     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
9     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
10    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
11    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
12    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
13          [1, 1, 1, 1, 1, 1, 1, 1]
14    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
15    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
16    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
17    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
19    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
dtype: object

# Evaluate the predictions

In [12]:
ap,first_pred_acc = evaluate(preds.tolist(), ground_truth.tolist())

In [13]:
print('average precision: {}'.format(ap))
print('first prediction accuracy: {}'.format(first_pred_acc))

average precision: 0.5719362003638895
first prediction accuracy: 0.7784368036426362


# Do recursive evaluation

In [14]:
for i in range(1,n_trk+1):
    print('------------ {} ----------------'.format(i))
    preds = pred_series_of_lists([l_prob[i_]>0.50 for i_ in range(i)], ground_truth.apply(len), i_2fill=-2)
    ap,first_pred_acc = evaluate(preds, ground_truth)
    print('average precision: {:.4f}'.format(ap))
#     print('first prediction accuracy: {}'.format(first_pred_acc))

------------ 1 ----------------
average precision: 0.5543
------------ 2 ----------------
average precision: 0.5624
------------ 3 ----------------
average precision: 0.5654
------------ 4 ----------------
average precision: 0.5690
------------ 5 ----------------
average precision: 0.5719
